In [2]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
print("[*] preprocessing")

CFG = {
    'IMG_SIZE':224,
    'EPOCHS':50,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

# all_img_list = glob.glob('./train/*/*')

# df = pd.DataFrame(columns=['img_path', 'label'])
# df['img_path'] = all_img_list
# # print(df['img_path'])
# # print(str(df['img_path'][0]).split('\\'))
# df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[1])

DATA_DIR = './data'

train_df = pd.read_csv(os.path.join(DATA_DIR, 'train_df_add_data.csv'))
train, val, _, _ = train_test_split(train_df, train_df['label'], test_size=0.3, stratify=train_df['label'], random_state=CFG['SEED'])

le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])
print(train.shape)

[*] preprocessing
(5170, 5)


In [4]:
print("[*] datasets ready")


class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        img_ar = np.fromfile(img_path, np.uint8)
        image = cv2.imdecode(img_ar, cv2.IMREAD_UNCHANGED)

#         image = cv2.imread(img_path)

        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)

# Augmentation는 rotation 과 vertical flip은 오히려 성능이 떨어지는 문제가 발생해서 제외하였습니다. 
# train_transform = A.Compose([
#                             A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
#                             A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), 
#                                         max_pixel_value=255.0, always_apply=False, p=1.0),
#                             ToTensorV2()
#                             ])

# # dy 방법
# train_transform = A.Compose([
#                             A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
#                             A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), 
#                                         max_pixel_value=255.0, always_apply=False, p=1.0),
#                             A.HorizontalFlip(p=0.5),
#                             A.OneOf([
#                                     A.IAAAdditiveGaussianNoise(),
#                                     A.GaussNoise(),
#                                 ], p=0.2),
#                             A.OneOf([
#                                     A.OpticalDistortion(p=0.3),
#                                     A.GridDistortion(p=.1),
#                                     A.IAAPiecewiseAffine(p=0.3),
#                                 ], p=0.2),
#                             A.OneOf([
#                                 A.IAASharpen(),
#                                 A.IAAEmboss(),
#                                 A.RandomBrightnessContrast(),            
#                                 ], p=0.3),
#                             ToTensorV2()
#                             ])

# 방법 1
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), 
                                        max_pixel_value=255.0, always_apply=False, p=1.0),
                            A.HorizontalFlip(p=0.5),
                            A.RandomRotate90(),
                            A.Flip(),  # 순서 뒤집기
                            A.Transpose(),  # 전치 행렬
                            A.OneOf([ # 함수 내부에서 1개를 p의 확률로 선택해주는 함수
                                    A.IAAAdditiveGaussianNoise(),
                                    A.GaussNoise(),
                                ], p=0.2),
                            A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
                            A.OneOf([
                                    A.OpticalDistortion(p=0.3),
                                    A.GridDistortion(p=.1),
                                    A.IAAPiecewiseAffine(p=0.3),
                                ], p=0.2),
                            A.OneOf([
                                A.IAASharpen(),
                                A.IAAEmboss(),
                                A.RandomBrightnessContrast(),            
                                ], p=0.3),
                            A.HueSaturationValue(p=0.3),
                            ToTensorV2()
                            ])

# # # 방법 2               
# train_transform = A.Compose([
#                             A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
#                             A.HorizontalFlip(p=0.5),  # 좌우반전
#                             A.RandomBrightnessContrast(p=0.5),
#                             A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
#                             ToTensorV2()
#                             ])
# # 방법 3
# train_transform = A.Compose([
#                             A.ElasticTransform(),
#                             A.GridDistortion(),
#                             A.HorizontalFlip(p=0.5),  # 좌우반전
#                             A.OpticalDistortion(),
#                             A.RandomBrightnessContrast(p=0.5),
#                             A.ShiftScaleRotate(),  # shift, scale, rotate 를 무작위
#                             A.VerticalFlip(p=0.5)  # 상하반전
#                             ])


test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), 
                                        max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

[*] datasets ready


In [5]:
print("[*] Model Define")


class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_v2_l(pretrained=True)
#         self.backbone = models.densenet161(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

[*] Model Define


In [6]:
# FocalLoss_DACON
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha]*19)
        self.alpha[18] = 1-alpha
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * at

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

In [7]:
# class FocalLoss(torch.nn.Module):
#     def __init__(self, alpha=1, gamma=2, reduction="mean"):
#         super(FocalLoss, self).__init__()
#         self.alpha = alpha
#         self.gamma = gamma
#         self.reduction = reduction

#     def forward(self, inputs, targets):
#         BCE_loss = F.cross_entropy(inputs, targets, reduction="none")
#         pt = torch.exp(-BCE_loss)
#         F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

#         if self.reduction == "mean":
#             return torch.mean(F_loss)
#         elif self.reduction == "sum":
#             return torch.sum(F_loss)
#         else:
#             return F_loss

In [8]:
print("[*] Train")

def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
#     criterion = nn.CrossEntropyLoss().to(device)
    criterion = FocalLoss(gamma=2, alpha=0.25)

    best_score = 0
    best_model = None

    for epoch in range(1, CFG['EPOCHS'] + 1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)      # ADDED .type(torch.LongTensor)

            optimizer.zero_grad()

            output = model(imgs)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(
            f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')

        if scheduler is not None:
            scheduler.step(_val_score)

        if best_score < _val_score:
            best_score = _val_score
            best_model = model

    return best_model, best_score


def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)      # ADDED .type(torch.LongTensor)

            pred = model(imgs)

            loss = criterion(pred, labels)

            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()

            val_loss.append(loss.item())

        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')

    return _val_loss, _val_score



[*] Train


In [9]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5,
                                                       patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model, f1_score = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/162 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.51249] Val Loss : [0.21143] Val Weighted F1 Score : [0.60851]


  0%|          | 0/162 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

...

  0%|          | 0/70 [00:00<?, ?it/s]

Epoch [49], Train Loss : [0.17037] Val Loss : [0.05124] Val Weighted F1 Score : [0.92889]


  0%|          | 0/162 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

Epoch [50], Train Loss : [0.17625] Val Loss : [0.05367] Val Weighted F1 Score : [0.92688]


In [10]:
print("[*] Inference")

test = pd.read_csv('./data/test.csv')

test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

[*] Inference


In [11]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)

            preds = model(imgs)
#             print(preds)
            
            preds = preds.argmax(1)
            preds = preds.cpu().detach().numpy()
            predictions += preds.tolist()
#             preds += preds.argmax(1).detach().cpu().numpy().tolist()

#             probs = probs.cpu().detach().numpy()
#             preds = probs > 0.5
#             preds = preds.astype(int)
#             predictions += preds.tolist()
    prediction = le.inverse_transform(preds)
    return predictions

In [12]:
preds = inference(model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

In [13]:
print("[*] Submission")
submit = pd.read_csv('./data/sample_submission.csv')
submit['label'] = preds
submit.head()


[*] Submission


,id,label
0,TEST_000,18
1,TEST_001,10
2,TEST_002,18
3,TEST_003,18
4,TEST_004,10


In [14]:
submit.loc[submit['label'] == 0, 'label'] = '가구수정'
submit.loc[submit['label'] == 1, 'label'] = '걸레받이수정'
submit.loc[submit['label'] == 2, 'label'] = '곰팡이'
submit.loc[submit['label'] == 3, 'label'] = '꼬임'
submit.loc[submit['label'] == 4, 'label'] = '녹오염'
submit.loc[submit['label'] == 5, 'label'] = '들뜸'
submit.loc[submit['label'] == 6, 'label'] = '면불량'
submit.loc[submit['label'] == 7, 'label'] = '몰딩수정'
submit.loc[submit['label'] == 8, 'label'] = '반점'
submit.loc[submit['label'] == 9, 'label'] = '석고수정'
submit.loc[submit['label'] == 10, 'label'] = '오염'
submit.loc[submit['label'] == 11, 'label'] = '오타공'
submit.loc[submit['label'] == 12, 'label'] = '울음'
submit.loc[submit['label'] == 13, 'label'] = '이음부불량'
submit.loc[submit['label'] == 14, 'label'] = '창틀,문틀수정'
submit.loc[submit['label'] == 15, 'label'] = '터짐'
submit.loc[submit['label'] == 16, 'label'] = '틈새과다'
submit.loc[submit['label'] == 17, 'label'] = '피스'
submit.loc[submit['label'] == 18, 'label'] = '훼손'

In [15]:
submit.head()

,id,label
0,TEST_000,훼손
1,TEST_001,오염
2,TEST_002,훼손
3,TEST_003,훼손
4,TEST_004,오염


In [16]:
submit.to_csv('./results/effv2_l_macro'+str(f1_score) + '.csv', index=False, encoding="utf-8-sig")